<a href="https://colab.research.google.com/github/NaimMhedhbi1/Machine-Learning/blob/master/the_final_model_arabic_texts_bert16_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

Saving dialect tunisian corpus for sentiment Analysis with 3 classes.txt to dialect tunisian corpus for sentiment Analysis with 3 classes.txt


In [2]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['dialect tunisian corpus for sentiment Analysis with 3 classes.txt']), lineterminator='\n')

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 172kB/s 
     |████████████████████████████████| 6.8MB 49.6MB/s 
     |████████████████████████████████| 983kB 36.4MB/s 
     |████████████████████████████████| 266kB 34.4MB/s 
     |████████████████████████████████| 1.3MB 29.9MB/s 
     |████████████████████████████████| 1.2MB 28.1MB/s 
     |████████████████████████████████| 471kB 46.3MB/s 
     |████████████████████████████████| 890kB 48.3MB/s 
     |████████████████████████████████| 2.9MB 41.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=84673a84b7bdb9eb087bdac84ef44835ce7e0181db23789a40b6ff63e00b65ec
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=7d069e71e0049177ac9ef05494048346a7208a3695f5d8acb5d7c3df47a0f559
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

**Importing required libraries:**

In [ ]:
import pandas as pd
import numpy as np
import ktrain
import tensorflow as tf
from ktrain import text

Checking version of TF, so that when later we will reload the model, we can use same version of TF:

In [ ]:
tf.version.VERSION

# Preparing our train and test dataset:

Get data loaded from train.txt to df_train dataframe and teast.txt to df_test dataframe.

In [ ]:
#df = pd.read_csv("../input/tunisian-texts/dialect tunisian corpus for sentiment Analysis with 3 classes.txt",lineterminator='\n')
#,lineterminator='\n'

In [ ]:
df['label'] = df['label'].astype(str)
df['text'] = df['text'].astype(str)

In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df , test_size=0.1, random_state=42)

Checking first few rows of our train dataset.

In [ ]:
df_train.head()

Checking category wise distribution of our test data.

In [ ]:
df_train.label.value_counts()

In [ ]:
X_train = df_train.text.tolist()
X_test = df_test.text.tolist()
y_train = df_train.label.tolist()
y_test = df_test.label.tolist()

Checking size of our train and test datasets:

In [ ]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

Our dataset has below categories/factors:

In [ ]:
factors = ['0', '1','2']

Encoding our sentiment categories into numeric values:

In [ ]:
encoding = { '0': 0,
    '1': 1,
'2' : 2   
}

In [ ]:
y_train = [encoding[key] for key in y_train]
y_test = [encoding[key] for key in y_test]

# Building Model using Transformer

We are using bert-base-uncased model. You can choose any other model. I am selecting maxlen of tokenization as 512 (it's max for BERT).

In [ ]:
model_arch ='bert-base-uncased'
MAXLEN = 100
trans = text.Transformer(model_arch, maxlen=MAXLEN, class_names= factors)


Let's preprocess out test and train data set.

In [ ]:
train_data = trans.preprocess_train(X_train,y_train)
test_data = trans.preprocess_test(X_test,y_test)

In [ ]:
model = trans.get_classifier()

In [ ]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=32)

Finding the best learning rate:

https://www.kaggle.com/search?q=bert+in%3Anotebooks+date%3A90

In [ ]:
#learner.lr_find(show_plot=True, max_epochs=4)

In [ ]:
learner.fit_onecycle(1e-4, 10)
#learner.autofit(0.0001,10)

**Confusion Matrix:**

In [ ]:
learner.model.summary()

In [ ]:
learner.validate(val_data=test_data, class_names=factors)

**top 5 data points not performing good:**

In [ ]:
learner.view_top_losses(n=5, preproc=trans)

In [ ]:
X_test[1928]

Above data our model is predicting as sadness but label is mentioned as fear.

# Predict Data:

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
inp = 'Makhyebkom w makhyeb serviskom 9adsh manhebkomch'

In [ ]:
predictor.predict(inp)

In [ ]:
!pip3 install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [ ]:
predictor.explain(inp)

In [ ]:
predictor.predict("الغرفة كانت نظيفة ، الطعام ممتاز ، وأنا أحب المنظر من غرفتي.")


In [ ]:
predictor.predict("mahlekooooom ya jme3a 9adesh el service mte3kom 5ayebbbb")

In [ ]:
predictor.predict("vraiment mouch normal haja tayarra barcha wallahi yaatikom alf al f sahha .")

In [ ]:
predictor.predict("Makhyeb eeichetkom 9adesh nakrahha ")

In [ ]:
predictor.predict("yaatikom essahaha haja wowwww")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
# save model for later use
predictor.save('/content/drive/MyDrive/Data Science Project Naim /arabic_predictor')
# save model using transformers API after partially training
learner.model.save_pretrained('/content/drive/MyDrive/Data Science Project Naim /arabic_predictor')


In [ ]:
# reload from disk
loaded_model = ktrain.load_predictor('/content/drive/MyDrive/Data Science Project Naim /arabic_predictor')

In [ ]:
# still works as expected after reloading from disk
t1 = 'aka rihab mahich professionel  w khaletha bina ya sahbi fi chtar ethniyyaa brass la7nina chnowa rayek '
loaded_model.predict(t1)

In [ ]:
loaded_model.explain(t1)

In [ ]:
"""from sklearn.metrics import roc_auc_score
y_pred = learner.model.predict(X_test, verbose=0)
score = roc_auc_score(Y_test, y_pred)
print("\n ROC-AUC score: %.6f \n" % (score))"""

In [ ]:
# save model using transformers API after partially training
#learner.model.save_pretrained('/content/arabic_predictor')

# reload the model using transformers directly
from transformers import *
model = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Data Science Project Naim /arabic_predictor')
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# re-instantiate Learner and continue training
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=32)
learner.fit_onecycle(1e-4, 10)

In [ ]:
# save model using transformers API after partially training
learner.model.save_pretrained('/content/drive/MyDrive/Data Science Project Naim /arabic_predictor/my_model')

In [ ]:
predictor.save('/content/drive/MyDrive/Data Science Project Naim /arabic_predictor')

+++++++++++++++++++++++++++++++++++++++++++++++